In [1]:
 import os
import hdf5storage
import numpy as np
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import f_classif
# from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
# from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from pgmpy.estimators import PC
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
from scipy.stats import entropy
import os
import pandas as pd
from scipy.io import loadmat

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
"""
Load Data Seperate All data in Protective and Non-Prtective Data
"""

# Assuming the 'Data' folder is in the current working directory
data_folders = ['./Data']

# Initialize lists to store the separated dataframes
protective_dfs = []
non_protective_dfs = []

# Loop through each data folder
for data_folder in data_folders:
    # List all .mat files in the current data folder
    mat_files = [f for f in os.listdir(data_folder) if f.endswith('.mat')]
    # Load each mat file
    for mat_file in mat_files:
        # Construct the full path to the .mat file
        mat_path = os.path.join(data_folder, mat_file)
        # Load the .mat file
        mat_data = loadmat(mat_path)
        # Convert the data into a pandas dataframe
        df = pd.DataFrame(mat_data['data'])
        # Select only the first 70 columns and the last column (73rd) which contains the behavior label
        df = df.iloc[:, list(range(66)) + [70] + [72]]
        # Split the data based on the protective behavior label
        # Assuming the last column in df is the protective behavior label
        protective_behavior = df.iloc[:, -1]
        protective_df = df[protective_behavior == 1]
        non_protective_df = df[protective_behavior == 0]
        # Append the resulting dataframes to their respective lists
        protective_dfs.append(protective_df)
        non_protective_dfs.append(non_protective_df)

# Concatenate all protective and non-protective dataframes
all_protective_data = pd.concat(protective_dfs, axis=0, ignore_index=True)
all_non_protective_data = pd.concat(non_protective_dfs, axis=0, ignore_index=True)

# Now `all_protective_data` and `all_non_protective_data` hold the protective and non-protective data respectively
# You can process these dataframes as needed for your analysis or save them to new .mat files
print(all_protective_data.shape)
print(all_non_protective_data.shape)

(77298, 68)
(437247, 68)


In [3]:
nodes_data = {}

# There are 22 nodes, so we loop through each
for node in range(1, 23):
    # Calculate the index for x, y, and z based on the node number
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44

    # Extract the data for the current node
    node_data = all_non_protective_data.iloc[:, [x_index, y_index, z_index]]

    # Assign the node data to the corresponding entry in the dictionary
    nodes_data[f'node_{node}'] = node_data

In [4]:
 # (node_from, node_to) based on the known graph structure
edges = [
    (1, 2), (1, 5), (1, 8), (2, 3), (3, 4), (5, 6), (6, 7),
    (8, 9), (9, 10), (9, 15), (9, 20), (10, 11), (11, 12),
    (12, 13), (13, 14), (15, 16), (16, 17), (17, 18), (18, 19),
    (9, 20), (20, 21), (21, 22)
]

In [5]:
def calculate_kl_divergence(p, q, epsilon=1e-10):
    p_normed = (p + epsilon) / (np.sum(p) + epsilon * len(p))
    q_normed = (q + epsilon) / (np.sum(q) + epsilon * len(q))
    return entropy(p_normed, q_normed)

# Exercise Type Analysis

In [6]:
exercise_labels = {
    1: "One-leg-stand",
    2: "Reach-forward",
    3: "Bend",
    4: "Sit-to-stand",
    5: "Stand-to-sit",
    6: "Sitting still",
    7: "Standing still",
    8: "Walking",
    0: "Others"
}
def categorize_exercises(dataframe):
    # Assuming the penultimate column in dataframe is the Exercise Type
    exercise_type = dataframe.iloc[:, -2]
    categorized_data = {}
    for label, description in exercise_labels.items():
        # Filter the dataframe by each exercise type
        exercise_df = dataframe[exercise_type == label]
        # Store the filtered dataframe in a dictionary
        categorized_data[description] = exercise_df
    return categorized_data

categorized_protective = categorize_exercises(all_protective_data)
categorized_non_protective = categorize_exercises(all_non_protective_data)

# Bend

In [7]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Bend"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

In [8]:
def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:6113],
            labels[1]: node_data_2[i].iloc[:6113]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [9]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [10]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 8.71401432937379


In [11]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 8.713507375315494


In [12]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 8.714014329373788


In [13]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 8.714988272959458


In [14]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 8.715918521521544


In [15]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 8.715103910849574


In [16]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 8.715262504538268


In [17]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 8.714988272959678


In [18]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 8.716116432061426


In [19]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 8.716116432061426


In [20]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 8.71608284415671


In [21]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 8.716367861730049


In [22]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 8.716465948901657


In [23]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 8.71646594890311


In [24]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 8.716762803022371


In [25]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 8.716192347592463


In [26]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 8.716318602487462


In [27]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 8.71635653532511


In [28]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 8.71635653532511


In [29]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 8.716209140526738


In [30]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 8.716247085824001


# Sit to Stand

In [31]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Sit-to-stand"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:7735],
            labels[1]: node_data_2[i].iloc[:7735]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [32]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [33]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 8.942255262524787


In [34]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 8.94191798867209


In [35]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 8.942255262524787


In [36]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 8.942877149522113


In [37]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 8.943834948534622


In [38]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 8.942877149518395


In [39]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 8.942877149518585


In [40]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 8.942877149520564


In [41]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 8.947220314676768


In [42]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 8.947220314676768


In [43]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 8.94544366417302


In [44]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 8.948554302265654


In [45]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 8.949957153963673


In [46]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 8.95047083857991


In [47]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 8.951158739298803


In [48]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 8.948046991685525


In [49]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 8.949551973729694


In [50]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 8.949951681902508


In [51]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 8.950780687733568


In [52]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 8.949205353553433


In [53]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 8.950081906066268


# Stand to Sit

In [54]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Stand-to-sit"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:10343],
            labels[1]: node_data_2[i].iloc[:10343]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [55]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [56]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 9.240484265230426


In [57]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 9.240226250382278


In [58]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 9.240484265230426


In [59]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 9.241068156785056


In [60]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 9.241165444842748


In [61]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 9.241120480586885


In [62]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 9.241165444842748


In [63]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 9.241068156784175


In [64]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 9.242101550444817


In [65]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 9.242101550444817


In [66]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 9.242071786932433


In [67]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 9.242510364376075


In [68]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 9.243032918907856


In [69]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 9.243234287191264


In [70]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 9.243234287191264


In [71]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 9.242168820096452


In [72]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 9.242878289129807


In [73]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 9.24310516650013


In [74]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 9.243137451361463


In [75]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 9.242439881731661


In [76]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 9.24268533794766


# Sitting Still

In [77]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Sitting still"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:3968],
            labels[1]: node_data_2[i].iloc[:3968]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [78]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [79]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 7.174905814462284


In [80]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 7.172074490773202


In [81]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 7.174905814462282


In [82]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 7.179484750004505


In [83]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 7.236033302188903


In [84]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 7.199047072213411


In [85]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 7.205066491001564


In [86]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 7.179484750004673


In [87]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 7.50019945434043


In [88]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 7.500199454340438


In [89]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 7.17948475000443


In [90]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 7.792994096739456


In [91]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 7.914538608284116


In [92]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 7.937839359204697


In [93]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 8.075806428200403


In [94]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 7.570064348185162


In [95]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 7.8721430376426325


In [96]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 7.893558353273481


In [97]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 8.045102435193167


In [98]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 7.879022446734749


In [99]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 7.947430042901249


# Standing Still

In [100]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Standing still"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:2174],
            labels[1]: node_data_2[i].iloc[:2174]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [101]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [102]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 6.732349485713726


In [103]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 6.723523660232641


In [104]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 6.732349485713722


In [105]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 6.75198747615735


In [106]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 6.766357686556636


In [107]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 6.752527852025459


In [108]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 6.753182878875935


In [109]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 6.751987476157396


In [110]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 6.768939542502228


In [111]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 6.768939542502228


In [112]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 6.751987476157507


In [113]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 6.914587385237032


In [114]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 7.129447333788197


In [115]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 7.200754058412421


In [116]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 7.401434779888592


In [117]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 6.781990437042595


In [118]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 6.971959545842264


In [119]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 7.056369339626863


In [120]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 7.442772550143565


In [121]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 7.073280630708324


In [122]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 7.2034328132016485


# Walking

In [123]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Walking"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:7554],
            labels[1]: node_data_2[i].iloc[:7554]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [124]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [125]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 8.92778248899995


In [126]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 8.927335917566412


In [127]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 8.92778248899995


In [128]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [129]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [132]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 8.928812129433329


In [130]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 8.928842784718796


In [133]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 8.928798562190424


In [134]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [135]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [136]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [137]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [138]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [139]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [140]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [141]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [142]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [143]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [144]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [145]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

In [146]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Node not present in the Directed Graph

# Others

In [147]:
nodes_data = {}

for node in range(1, 23):
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44
    node_data = categorized_non_protective["Others"].iloc[:, [x_index, y_index, z_index]]
    nodes_data[f'node_{node}'] = node_data

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:16135],
            labels[1]: node_data_2[i].iloc[:16135]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [148]:
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [149]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_1, Final weight: 9.522969998429758


In [150]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 9.509374130900545


In [151]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_1, Final weight: 9.52296999842975


In [152]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_2_to_node_3, Final weight: 9.550117452048037


In [153]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_4, Final weight: 9.55644370334261


In [154]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_5, Final weight: 9.557356823807275


In [155]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 9.56178418496374


In [156]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 9.550117452048843


In [157]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_10_to_node_9, Final weight: 9.57554398790657


In [158]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 9.57554398790657


In [159]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_20, Final weight: 9.550117452047084


In [160]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 9.606092366729584


In [161]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_11, Final weight: 9.630401299637121


In [162]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 9.6373246701881


In [163]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_14_to_node_13, Final weight: 9.656502883682927


In [164]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 9.586224019551496


In [165]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_16, Final weight: 9.62575921285219


In [166]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_17_to_node_18, Final weight: 9.63281200423869


In [167]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_19_to_node_18, Final weight: 9.658288611906526


In [168]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_20, Final weight: 9.620825920069953


In [169]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 9.633782010628046
